# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

    Name:Vladislav Mun

    Student ID: 16BD02178

    Email:vlad.mun1999@gmail.com

In [32]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [33]:
data = pd.read_csv("2008.csv.bz2")
data.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.0,7.009728e+06,7.009728e+06,7.009728e+06,6.873482e+06,7.009728e+06,6.858079e+06,7.009728e+06,7.009728e+06,6.855029e+06,...,7.009728e+06,6.858079e+06,6.872670e+06,7.009728e+06,7.009728e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06
mean,2008.0,6.375130e+00,1.572801e+01,3.924182e+00,1.333830e+03,1.326086e+03,1.481258e+03,1.494801e+03,2.224200e+03,1.273224e+02,...,7.263870e+02,6.860852e+00,1.645305e+01,1.960618e-02,2.463006e-03,1.577206e+01,3.039031e+00,1.716462e+01,7.497434e-02,2.077098e+01
std,0.0,3.406737e+00,8.797068e+00,1.988259e+00,4.780689e+02,4.642509e+02,5.052251e+02,4.826728e+02,1.961716e+03,7.018731e+01,...,5.621018e+02,4.933649e+00,1.133280e+01,1.386426e-01,4.956753e-02,4.009912e+01,1.950287e+01,3.189495e+01,1.837940e+00,3.925964e+01
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.200000e+01,...,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,6.220000e+02,7.700000e+01,...,3.250000e+02,4.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.320000e+03,1.512000e+03,1.517000e+03,1.571000e+03,1.100000e+02,...,5.810000e+02,6.000000e+00,1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.907000e+03,3.518000e+03,1.570000e+02,...,9.540000e+02,8.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.600000e+01
max,2008.0,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.400000e+03,9.743000e+03,1.379000e+03,...,4.962000e+03,3.080000e+02,4.290000e+02,1.000000e+00,1.000000e+00,2.436000e+03,1.352000e+03,1.357000e+03,3.920000e+02,1.316000e+03


In [34]:
data['CancellationCode'].value_counts() #weather B (1)

B    54904
A    54330
C    28188
D       12
Name: CancellationCode, dtype: int64

In [35]:
data["Distance"].max() #(2) #max = 4962 miles

4962

In [36]:
data["Distance"].min() #(2) # min = 11 miles 

11

In [37]:
data["Distance"].mean()#(2) #726 miles avg

726.3870294253928

In [39]:
mindist = data.loc[data['Distance'] == data["Distance"].min()]#(3); 2 flights flew for 11 miles (4988 and 5572)
mindist

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2547298,2008,5,15,4,2037.0,1800,2125.0,1900,OH,4988,...,10.0,7.0,0,NaN,0,145.0,0.0,0.0,0.0,0.0
4392215,2008,8,10,7,1315.0,1220,1415.0,1320,OH,5572,...,8.0,38.0,0,NaN,0,55.0,0.0,0.0,0.0,0.0


In [8]:
array = ['4988', '5572']
a = data.loc[(data['UniqueCarrier'] == 'OH') & data['FlightNum'].isin(array)]
a #(3) a list of all the flights with the numbers mentioned before, and the same carrier

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
151629,2008,1,2,3,1005.0,945,1213.0,1149,OH,5572,...,8.0,39.0,0,NaN,0,0.0,20.0,4.0,0.0,0.0
151771,2008,1,3,4,1700.0,1550,1945.0,1815,OH,4988,...,11.0,32.0,0,NaN,0,0.0,0.0,20.0,0.0,70.0
152291,2008,1,3,4,955.0,938,1228.0,1142,OH,5572,...,8.0,51.0,0,NaN,0,17.0,0.0,29.0,0.0,0.0
152430,2008,1,4,5,1542.0,1550,1817.0,1815,OH,4988,...,10.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
152949,2008,1,4,5,938.0,938,1130.0,1142,OH,5572,...,9.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
153088,2008,1,5,6,1550.0,1550,1813.0,1815,OH,4988,...,8.0,13.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
153419,2008,1,5,6,938.0,938,1127.0,1142,OH,5572,...,7.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
153545,2008,1,6,7,1550.0,1550,1830.0,1815,OH,4988,...,10.0,19.0,0,NaN,0,0.0,0.0,15.0,0.0,0.0
153963,2008,1,6,7,938.0,938,1148.0,1142,OH,5572,...,8.0,37.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
154096,2008,1,7,1,1540.0,1550,1810.0,1815,OH,4988,...,15.0,12.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [9]:
a["Distance"].value_counts() #(3) the distances the flights flew

544    243
834    130
11       2
96       1
Name: Distance, dtype: int64

In [10]:
 data['Origin'].value_counts()#William B Hartsfield-Atlanta Intl - most flights start #(4)

ATL    414513
ORD    350380
DFW    281281
DEN    241443
LAX    215608
PHX    199408
IAH    185172
LAS    172876
DTW    161989
SFO    140587
SLC    139088
EWR    138506
MCO    130872
MSP    130289
CLT    126045
LGA    119135
JFK    118804
BOS    117915
SEA    109069
BWI    104074
PHL    100499
SAN     93775
CVG     91265
MDW     87619
DCA     86662
MEM     80966
TPA     78179
IAD     76031
FLL     67177
CLE     65795
        ...  
OTH       515
LMT       511
SLE       486
ACK       457
STX       436
MKG       396
YKM       340
ALO       323
CMX       318
RFD       290
BPT       276
PLN       264
WYS       264
RHI       221
TEX       194
LWB       185
HTS       158
ITH       132
SUX       122
BLI       120
ACY       116
DLG       116
AKN       116
ADK       102
GST        85
BJI        74
INL        72
TUP        10
PIR         5
PUB         2
Name: Origin, Length: 303, dtype: int64

In [11]:
airports = data.groupby('Origin').agg('mean')
airtime = airports.loc[airports['AirTime'] == airports["AirTime"].max()]
airtime = airtime.reset_index()
airtime[['Origin','AirTime']] #Luis Munoz Marin International -longest airtime 204 miles

,Origin,AirTime
0,SJU,203.850843


In [12]:
airports = data.groupby('Origin').filter(lambda x: len(x) >= 1000)
airports =  airports.groupby('Origin')
delays = airports.DepDelay.agg(lambda x: (x > 0).sum() / len(x)) #delayed flight / total flights in this airport
delays.loc[delays == delays.max()] #dallas airport

Origin
DAL    0.578642
Name: DepDelay, dtype: float64

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [40]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [41]:
data.shape

(32769, 10)

In [42]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [43]:
# число значений у признаков
for col_name in data.columns:
    print (col_name, len(data[col_name].unique()))

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [44]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [45]:
import sklearn.neighbors as sn
import sklearn.metrics as sm
def match_indicator_metric(x, y):
    return (x != y).sum()

clf = sn.KNeighborsClassifier(metric='euclidean', n_neighbors=10)
clf.fit(X_train, y_train)
y_score = clf.predict_proba(X_test)
sm.roc_auc_score(y_test, y_score[:,1])

0.6794653376665294

In [46]:
clf = sn.KNeighborsClassifier(metric='manhattan', n_neighbors=10)
clf.fit(X_train, y_train)
y_score = clf.predict_proba(X_test)
sm.roc_auc_score(y_test, y_score[:,1])


0.7223722810596633

In [47]:
clf = sn.KNeighborsClassifier(metric='chebyshev', n_neighbors=10)
clf.fit(X_train, y_train)
y_score = clf.predict_proba(X_test)
sm.roc_auc_score(y_test, y_score[:,1])

0.6640882604748257

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

In [48]:
a = np.arange(1, 20, 1);
max = 0;
k = 1;
for i in a:
    clf = sn.KNeighborsClassifier(metric='euclidean', n_neighbors=i)
    clf.fit(X_train, y_train)
    y_score = clf.predict_proba(X_test)
    score = sm.roc_auc_score(y_test, y_score[:,1])
    if score > max:
        max = score
        k= i
score, k

(0.6823514094029204, 17)

In [49]:
a = np.arange(1, 20, 1);
max = 0;
k = 1;
for i in a:
    clf = sn.KNeighborsClassifier(metric='manhattan', n_neighbors=i)
    clf.fit(X_train, y_train)
    y_score = clf.predict_proba(X_test)
    score = sm.roc_auc_score(y_test, y_score[:,1])
    if score > max:
        max = score
        k= i
score, k

(0.716093262196459, 10)

In [50]:
a = np.arange(1, 20, 1);
max = 0;
k = 1;
for i in a:
    clf = sn.KNeighborsClassifier(metric='chebyshev', n_neighbors=i)
    clf.fit(X_train, y_train)
    y_score = clf.predict_proba(X_test)
    score = sm.roc_auc_score(y_test, y_score[:,1])
    if score > max:
        max = score
        k= i
score, k

(0.6602992299016405, 16)

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

Здесь вы можете поделиться своими мыслями о задании.

А здесь вставьте смешную картинку.

А здесь посоветуйте преподавателям хороший фильм или сериал.